In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib widget
#%matplotlib ipympl

#%reload_ext tensorboard
#%matplotlib qt

In [ ]:
import pandas as pd
import os, pickle
import numpy as np
from pathlib import Path
import SimpleITK as sitk
import pydicom
from LN_malignancy_GNN.pytorch.run_model_torch import RunModel

In [ ]:
data_path = Path('../../data/ln_malignancy')
fold_dir = data_path.joinpath('liyuan_folds')
dicom_dir = data_path.joinpath('All_Pt_Data')

In [ ]:
fold_dict = {}
patient_list = []

In [ ]:
for idx in range(5):
    fold_dict[f"fold_{idx}"] = np.unique([f.name.split('_')[0] for f in fold_dir.glob(f"F{idx+1}/label/*")])
    patient_list.extend(np.unique([f.name.split('_')[0] for f in fold_dir.glob(f"F{idx+1}/label/*")]))

In [ ]:
preset_fold_list = []
for fold in fold_dict:
    preset_fold_list.append([patient_list.index(pat) for pat in fold_dict[fold]])

In [ ]:
with open(data_path.joinpath('preset_folds_malignancy_liyuan_091024.pkl'), 'wb') as f:
    pickle.dump(preset_fold_list, f)
    f.close()

In [ ]:
fold_file_ex = pd.read_pickle(data_path.joinpath('preset_folds_malignancy_081324.pkl'))

In [ ]:
patients = pd.read_pickle(data_path.joinpath('features_and_labels_070924.pkl'))

In [ ]:
list(patients.index.get_level_values(0).unique()) == patient_list

In [ ]:
from LN_malignancy_GNN.pytorch.gen_params_torch_cfg import *
model_config['data_version'] = 'initial_rot5_30_balance_v1'
model_config['balance_classes'] = True

In [ ]:
test_model = RunModel(model_config)

In [ ]:
test_model.set_data()

In [ ]:
test_model.set_train_test_split()

In [ ]:
test_model.data.processed_file_names

In [ ]:
n_pats = len(test_model.data.patients.index.get_level_values(0).unique())

test_idx = [ patients.index.get_level_values(0).unique().get_loc(pat.split('_')[0])+ n_pats *int(pat.split('_')[-1]) for pat in aug_fold_pats]

In [ ]:
for pat, idx in zip(aug_fold_pats, test_idx):
    print(pat, idx)

In [ ]:
len(test_model.data.patients.iloc[test_model.train_splits[0]].index.get_level_values(0).unique())

In [ ]:
train_fold_pats = test_model.data.patients.iloc[test_model.train_splits[0]]

In [ ]:
fold_pats = train_fold_pats.index.get_level_values(0).unique()

In [ ]:
aug_idx = [file_name.split('_')[1] for file_name in test_model.data.processed_file_names if file_name.split('_')[2] in fold_pats and 'rotation' in file_name]

In [ ]:
all_patients = list(test_model.data.patients.index.get_level_values(0).unique()) + list(test_model.data.aug_patients.index.get_level_values(0).unique()) + list(test_model.data.aug_pos_patients.index.get_level_values(0).unique())

In [ ]:
test_str = 'graph_1429_92493024_pos_rotation_2.pt'

In [ ]:
'_'.join(test_str.split('.')[0].split('_')[2:])

In [ ]:
fold_pats = test_model.data.patients.index.get_level_values(0).unique()[test_model.train_splits[0]]
print(fold_pats)

In [ ]:
for idx in test_model.train_splits[0]:
    print(idx, all_patients[idx])

In [ ]:
for idx in:
    L.seed_everything(42)
    model.set_model()
    model.set_callbacks(idx)
    
    model.trainers[idx] = L.Trainer(
                    max_epochs=model.config['n_epochs'],
                    accelerator="auto",
                    devices=model.config['gpu_device'] if torch.cuda.is_available() else None,
                    logger=[L.loggers.CSVLogger(save_dir=os.path.join(model.log_dir, f"csvlog_fold_{idx}")), L.loggers.TensorBoardLogger(save_dir=os.path.join(model.log_dir, f"tb_fold_{idx}"))],
                    callbacks=model.callbacks,
                    #check_val_every_n_epoch = 1,
                    #auto_lr_find=True
    )
    model.trainers[idx].fit(model.model, datamodule=model.data_module_cross_val[idx])

In [ ]:
idx = 3
model.best_checkpoints[f"fold_{idx}"] = {callback.monitor: callback.best_model_path for callback in model.trainers[idx].callbacks if 'ModelCheckpoint' in callback.__class__.__name__}
for monitor, best_model in model.best_checkpoints[f"fold_{idx}"].items():
                model.val_metrics[monitor][idx] = model.trainers[idx].validate(model.trainers[idx].model, datamodule=model.data_module_cross_val[idx], ckpt_path=best_model)[0]
                model.test_metrics[monitor][idx] = model.trainers[idx].test(model.trainers[idx].model, datamodule=model.data_module_cross_val[idx], ckpt_path=best_model)[0]

In [ ]:
test_arr = []
for idx in range(5):
    test_file = scipy.io.loadmat(model.data.data_path.joinpath(f'Radiomics_features/Radio F{idx+1}_ct_fea.mat').as_posix())
    tmp_arr = test_file['fin_fea_all']
    print(tmp_arr.shape)
    test_arr.append(tmp_arr)

In [ ]:
np.shape(test_arr)

In [ ]:
for fold in model.folds:
    print(len(fold))

In [ ]:
fold_1 = model.data.patients.loc[model.data.patients.index.get_level_values(0).unique()[model.folds[0]]]
print(fold_1)
print(test_arr[0][:, -6:])


In [ ]:
np.array([1,1]) + [3,1]

In [ ]:
pat_arr = model.data.patients['labels'].values
sum_arr = np.array([0,0])
for pat in pat_arr:
    sum_arr += pat

sum_arr

In [ ]:
order_dict = {value: key for key, value in enumerate(fold_1['dist_to_pri'])}

In [ ]:
ordered_test_arr = np.array(sorted(test_arr[0], key=lambda x: order_dict[x[-3]]))

In [ ]:
test_arr[1]

In [ ]:
len(np.unique(test_arr[1][:,-3]))

In [ ]:
len(np.rec.find_duplicate(test_arr[1][:, -2]))

In [ ]:
for idx, idx_str in enumerate(fold_1.index):
    try:
        print(fold_1.loc[idx_str, 'dist_to_pri'], ordered_test_arr[idx-1, -3])
    except:
        print('ran out of spaces')

In [ ]:
model.fold_dict = {}
for idx in range(5):
    model.fold_dict[f"fold_{idx}"] = [f.name.split('.')[0] for f in sorted(model.data.data_path.joinpath(model.config['fold_dir']).glob(f"F{idx+1}/label/*"))]

In [ ]:
model.fold_dict

In [ ]:
node_idx = {}
for idx in range(5):
    node_idx[f"fold_{idx}"] = [model.data.patients.index.get_loc((pat.split('_')[0], int(pat.split('_')[1]))) for pat in model.fold_dict[f"fold_{idx}"]]

In [ ]:
len(node_idx['fold_0'])

In [ ]:
import pickle
fold_dict = model.fold_dict
patients = model.data.patients
with open(model.data.data_path.joinpath('node_index_no_large.pkl'), 'wb') as f:
    pickle.dump(node_idx, f)
    pickle.dump(fold_dict, f)
    pickle.dump(patients, f)
    f.close()
    